In [8]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

'''
    ['0', 'Отзывы', 'f=154', 'HYUNDAI SOLARIS'],
    ['1', 'Двигатель', 'f=137', 'HYUNDAI SOLARIS'],
    ['2', 'Трансмиссия', 'f=143', 'HYUNDAI SOLARIS'],
    ['3', 'Ходовая', 'f=120', 'HYUNDAI SOLARIS'],
    ['4', 'Колеса', 'f=149', 'HYUNDAI SOLARIS'],
    ['5', 'Электрооборудование', 'f=140', 'HYUNDAI SOLARIS'],
    ['6', 'Мультимедиа', 'f=142', 'HYUNDAI SOLARIS'],
    ['7', 'Кузов', 'f=148', 'HYUNDAI SOLARIS'],
    ['8', 'Жидкости', '', 'HYUNDAI SOLARIS'],
    ['9', 'Сервис', 'f=19', 'HYUNDAI SOLARIS'],
    ['10', 'Эксплуатация', '', 'HYUNDAI SOLARIS']

    ['12', 'Кузов-Отопление', 'f=146', 'HYUNDAI SOLARIS'],
    ['13', 'Кузов-Охлаждение', 'f=147', 'HYUNDAI SOLARIS']    
'''

site = 'https://solaris-club.net/forum/'
folder_names = [
    ['11', 'Тюнинг', 'f=145', 'HYUNDAI SOLARIS']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [2]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [3]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}

        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])] = [author_id, author_nickname]
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        author = post_data[message_id][1]
                    else:
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            if item.has_attr('class') and item['class'][0] == 'smallfont':
                                post_date = item.get_text().strip()
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
        
    return messages

In [4]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [5]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [6]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

In [9]:
%%telegram_send Next message
scraper(site, folder_names)
'Information collected'

Каталог запчастей Hyundai Solaris 2017-2020
Сетка в бампер
Накладки на пороги
Чип-тюнинг
Органайзер в багажник, замена докатки на полноразмерное колесо
Подсветка Бардачка
Спойлер на багажник
Заменить бампер от другого рестайлинга
Фаркоп
Защита картера
Включение радиоканала ключа при заведеном авто
LED в задний бампер
Тюнинг-комплект QUANT на Солярис 2
Solaris 2020 выбор нового ГУ на android
Включение подогрева руля при Автозапуске и блокировка двигателя под капотом
Доп. установка штатного парктроника
Доустановить подушки безопасности: боковые, шторки
Подключение видеорегистратора.
Брызговики увеличенного размера 2017+
Нештатный подлокотник
Апгрэйд Соляриса
Каталог запчастей Hyundai Solaris 2017-2020
Недоложили гаечный ключ
Дополнительный свет
Установка багажника на крышу (седан)
Коврики для Solaris 2017
Дефлекторы на окна
Хромированные накладки на ДХО
Уплотнительное кольцо для парктроника
Круговой обзор
Взаимозаменяемость запчастей Солярис 1 - Solaris 2
Установка предпускового подогрев

'Information collected'

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [ ]:
folder_names = [['11', 'Тюнинг', 'f=145', 'HYUNDAI SOLARIS']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)